In [1]:
import eikon as ek

ek.set_app_key("69de0ce7c5e2468e90a6058ce79f6558ebe3ee30")

In [20]:
df = ek.get_timeseries(["MSFT.O"],
                       start_date="2016-01-01",
                       end_date="2019-01-10")
df

MSFT.O,HIGH,CLOSE,LOW,OPEN,COUNT,VOLUME
Date,,,,,,
2016-01-04,54.8,54.8,53.39,54.32,272781,53777963
2016-01-05,55.39,55.05,54.54,54.93,180637,34079674
2016-01-06,54.4001,54.05,53.64,54.32,225856,39518863
2016-01-07,53.485,52.17,52.07,52.7,303271,56564852
2016-01-08,53.28,52.33,52.15,52.37,261949,48753969
...,...,...,...,...,...,...
2019-01-04,102.51,101.93,98.93,99.72,300149,44060620
2019-01-07,103.2681,102.06,100.98,101.64,238930,35656136
2019-01-08,103.97,102.8,101.7134,103.04,223595,31514415


https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api/tutorials#fundamentals-api-rating-transition-matrix


In [31]:
ratings_data, err = ek.get_data(instruments=['0#.SPX'],
                                fields=['TR.CommonName',
                                        "TR.NAICSIndustryGroup",
                                        'TR.NumberOfAnalysts(SDate=0D,Period=FY1)',
                                        "TR.NumEstRevisingUp(Period=FY1)",
                                        "EARNINGS"])
ratings_data

,Instrument,Company Common Name,NAICS Industry Group Name,Number of Analysts,Number of Analysts Revising Up,EARNINGS
0,POOL.OQ,Pool Corp,Miscellaneous Durable Goods Merchant Wholesalers,11,0,19.5579
1,CHRW.OQ,CH Robinson Worldwide Inc,Freight Transportation Arrangement,29,1,7.2908
2,AJG.N,Arthur J. Gallagher & Co.,"Agencies, Brokerages, and Other Insurance Rela...",17,13,5.1985
3,CNP.N,CenterPoint Energy Inc,Natural Gas Distribution,17,2,1.4411
4,AMCR.N,Amcor PLC,Plastics Product Manufacturing,7,1,0.7116
...,...,...,...,...,...,...
498,DXC.N,DXC Technology Co,Computer Systems Design and Related Services,14,4,2.9352
499,SNPS.OQ,Synopsys Inc,Computer Systems Design and Related Services,16,2,6.2931
500,J.N,Jacobs Solutions Inc,"Architectural, Engineering, and Related Services",18,2,5.0228
501,SIVB.OQ,SVB Financial Group,Depository Credit Intermediation,23,7,25.3505


In [30]:
ratings_data.value_counts("NAICS Industry Group Name")

NAICS Industry Group Name
Lessors of Real Estate                               27
Electric Power Generation, Transmission and Di...    26
Navigational, Measuring, Electromedical, and C...    24
Insurance Carriers                                   21
Semiconductor and Other Electronic Component M...    18
                                                     ..
Electronics and Appliance Retailers                   1
Pulp, Paper, and Paperboard Mills                     1
Electrical Equipment Manufacturing                    1
Drycleaning and Laundry Services                      1
Promoters of Performing Arts, Sports, and Simi...     1
Length: 123, dtype: int64

# Timeseries

In [56]:
Hourly = ek.get_timeseries(["MSFT.O"], fields=["Close"], start_date = "2022-03-01", end_date = "2022-03-27", interval="hour")
Daily = ek.get_timeseries(["MSFT.O"], fields=["Close"], start_date = "2022-03-01", end_date = "2022-03-27", interval="daily")
Weekly = ek.get_timeseries(["MSFT.O"], fields=["Close"], start_date = "2022-03-01", end_date = "2022-03-27", interval="weekly")
Monthly = ek.get_timeseries(["MSFT.O"], fields=["Close"], start_date = "2022-03-01", end_date = "2022-05-27", interval="monthly")
Hourly.tail()

MSFT.O,CLOSE
Date,
2022-03-25 20:00:00,303.69
2022-03-25 21:00:00,303.45
2022-03-25 22:00:00,303.48
2022-03-25 23:00:00,303.51
2022-03-26 00:00:00,303.75


In [41]:
#a RIC that you want to get time series
RIC = 'IBM.N'
#start date that you want to get time series; it is in format yyyy-mm-dd
start_date = '2018-01-02'
#get time series till the last market close date using eikon.get_timeseries(..) function
df = ek.get_timeseries(RIC, 'CLOSE', interval='daily', start_date=start_date)
#display time series data frame
df

IBM.N,CLOSE
Date,
2018-01-02,147.334047
2018-01-03,151.383942
2018-01-04,154.450019
2018-01-05,155.204598
2018-01-08,156.140659
...,...
2023-02-06,136.18
2023-02-07,135.84
2023-02-08,135.98


In [51]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from datetime import timedelta, datetime
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.stattools import adfuller

#function to forecast using ARIMA Model
#input is time series data frame(df) got from eikon.get_timeseries(..)
#and forecast end date which is in format yyyy-mm-dd
def ARIMA_model_forecast(df, forecast_end_date):
    #Set the start and end forecast date time
    #first date is 1 day after time series. The last date is the forecast_end_date parameter
    _e_date = datetime.fromtimestamp(datetime.timestamp(df.index[-1]))
    e_date = _e_date + timedelta(days = 1)
    e_date = e_date.strftime('%Y-%m-%d')

    #find differences of time series which is input of ARIMA model
    df['df_1diff'] = df['CLOSE'].diff()
    df = df[df['df_1diff'].notnull()]
    lag_acf = acf(df['df_1diff'], nlags=50)
    lag_pacf = pacf(df['df_1diff'], nlags=50, method='ols')
    new_spot = df['df_1diff'].resample('D').ffill()
    #call ARIMA model which p=1,d=0 and q=1 with differences of time series
    arma_model = ARIMA(new_spot, order=(1,0,1))
    results = arma_model.fit(disp=0)
    residuals = pd.DataFrame(results.resid)
    #focast time series
    predictions_ARIMA = pd.Series(results.fittedvalues, copy=True)
    predictions_ARIMA_cumsum = predictions_ARIMA.cumsum()
    predictions_ARIMA_final = pd.Series(df['CLOSE'].iloc[0], index=new_spot.index)
    predictions_ARIMA_final = predictions_ARIMA_final.add(predictions_ARIMA_cumsum, fill_value=0)
    new_spot = df.CLOSE.resample('D',label='right').ffill()
    es_model = ExponentialSmoothing(new_spot, trend='add', damped=False, seasonal='mul', seasonal_periods=30)
    es_results = es_model.fit()
    predicted_values = es_model.predict(params=es_results.params, start=e_date, end=forecast_end_date)

    #create data frame from forecast timeseries
    preds = pd.DataFrame(index= pd.date_range(start=e_date, end=forecast_end_date), data=predicted_values, columns=['CLOSE'])

    #Plot graph of past and forecast timeseries
    plt.figure(figsize=(16,7))
    plt.plot(new_spot, label='Actual')
    plt.plot(preds, label='Forecast', color='red')
    plt.legend(loc='best')
    plt.show()
    return preds

#end date that you want to forecast
forecast_end_date = '2020-03-31'
#call the function to forecast time series with ARIMA model
forecast = ARIMA_model_forecast(df,forecast_end_date)
#display forecast time series
forecast

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data). The types seen wereNone and None. The data was
Date
2018-01-03    4.049895
2018-01-04    3.066076
2018-01-05     0.75458
2018-01-06     0.75458
2018-01-07     0.75458
                ...   
2023-02-06       -0.76
2023-02-07       -0.34
2023-02-08        0.14
2023-02-09       -2.23
2023-02-10        1.85
Freq: D, Name: df_1diff, Length: 1865, dtype: Float64
and
 [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
before. After,
[4.049895360000022 3.0660764399999607 0.7545795600000247 ...
 0.13999999999998636 -2.2299999999999898 1.8499999999999943]
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]].